# <font style="color:blue">Project 3: Object Detection</font>





#### Maximum Points: 100



|Sr. no.|Section|Points|
|:-------|:-------|:------:|
|1|Plot Ground Truth Bounding Boxes|20|
|2|Training|25|
|3|Inference|15|
|4|COCO Detection Evaluation|25|
|5|COCO Detection Evaluation|15|

# <font style="color:purple">Download the config file and trainer package</font> 

In [1]:
!wget "https://raw.githubusercontent.com/RadimKozl/OpenCV_PyTorch_Project3/refs/heads/main/trainer.zip" -O ./trainer.zip
!ls /kaggle/working/
!unzip /kaggle/working/trainer.zip
!rm /kaggle/working/trainer.zip

--2024-12-11 22:01:39--  https://raw.githubusercontent.com/RadimKozl/OpenCV_PyTorch_Project3/refs/heads/main/trainer.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32212 (31K) [application/zip]
Saving to: './trainer.zip'

./trainer.zip       100%[===================>]  31.46K  --.-KB/s    in 0.01s   

2024-12-11 22:01:40 (2.32 MB/s) - './trainer.zip' saved [32212/32212]

trainer.zip
Archive:  /kaggle/working/trainer.zip
   creating: trainer/.ipynb_checkpoints/
  inflating: trainer/.ipynb_checkpoints/configuration-checkpoint.py  
  inflating: trainer/.ipynb_checkpoints/datasets-checkpoint.py  
  inflating: trainer/.ipynb_checkpoints/hooks-checkpoint.py  
  inflating: trainer/.ipynb_checkpoints/matplotlib_visualizer-checkpoint.py  
  inflating: trainer/.i

In [2]:
!wget "https://github.com/RadimKozl/OpenCV_PyTorch_Project3/blob/main/config.yaml" -O ./config.yaml

--2024-12-11 22:01:44--  https://github.com/RadimKozl/OpenCV_PyTorch_Project3/blob/main/config.yaml
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: './config.yaml'

./config.yaml           [ <=>                ] 165.55K  --.-KB/s    in 0.1s    

2024-12-11 22:01:45 (1.59 MB/s) - './config.yaml' saved [169525]



# <font style="color:purple">Install external libraries</font> 

In [3]:
!pip install psutil
!pip install gpustat
!pip install orjson

In [4]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# <font style="color:purple">Imports libraries</font> 

In [5]:
import os
import gc
import json
import orjson
import csv
import yaml
import shutil
import pandas as pd
import numpy as np
import random
import time
import psutil
import gpustat
import matplotlib.pyplot as plt
import IPython
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

from operator import itemgetter
import multiprocessing as mp
mp.set_start_method('spawn', force=True)

# <font style="color:purple">Download the Dataset</font> 



**[Download the Vehicle registration plate](https://www.dropbox.com/s/k81ljpmzy3fgtx9/Dataset.zip?dl=1)**





Download the Vehicle Registration Plate dataset from [here](https://www.dropbox.com/s/k81ljpmzy3fgtx9/Dataset.zip?dl=1) and unzip it. 



We will have the following directory structure:



```

Dataset

├── train

│   └── Vehicle registration plate

│       └── Label

└── validation

    └── Vehicle registration plate

        └── Label

```



Unzipping the file will give you a directory `Dataset`. This directory has two folder `train` and `validation`. Each train and validation folder has `Vehicle registration plate`  folder with `.jpg` images and a folder `Labels`.  `Labels` folder has bounding box data for the images.





For example,

For image: `Dataset/train/Vehicle registration plate/bf4689922cdfd532.jpg`

Label file is  `Dataset/train/Vehicle registration plate/Label/bf4689922cdfd532.txt`



There are one or more lines in each `.txt` file. Each line represents one bounding box.

For example,

```

Vehicle registration plate 385.28 445.15 618.24 514.225

Vehicle registration plate 839.68 266.066462 874.24 289.091462

```



We have a single class detection (`Vehicle registration plate detection`) problem. So bounding box details start from the fourth column in each row.



Representation is in `xmin`, `ymin`, `xmax`, and `ymax` format.



**It has `5308` training and `386` validation dataset.**



Data is downloaded from [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html)

#  <font style="color:green">1. Plot Ground Truth Bounding Boxes [20 Points]</font> 



**You have to show three images from validation data with the bounding boxes.**



The plotted images should be similar to the following:



<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w10-g1.png'>







<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w10-g2.png'>







<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w10-g3.png'>




##  <font style="color:blue">1.1. Class for create JSON file of datasets</font>

We create Json structure of `train`, `valid` and `test` datasets, for creating PyTorch Dataloader a standard module of trainer.

In [6]:
class TransformDataset:
    def __init__(self, name_json_file='datasets.json'):
        self.name_json_file = name_json_file
        self.path_test_images = os.path.join('/kaggle', 'input', 'opencv-evalution-alpr-dataset', 'cars_ALPR_test')
        self.path_train_images = os.path.join('/kaggle', 'input', 'vehicle-registration-plate', 'Dataset', 'train', 'Vehicle registration plate')
        self.path_valid_images = os.path.join('/kaggle', 'input', 'vehicle-registration-plate', 'Dataset', 'validation', 'Vehicle registration plate')
        self.json_file = {
            "datasets": [
                {
                    "train": [],
                    "valid": [],
                    "test": [],
                    "class_number": 2,
                    "names_class": ['__background__', 'Vehicle registration plate']
                }
            ]
        }

    def __select_files(self, root_path, endswith_file):
        return [os.path.join(root_path, filename) for filename in os.listdir(root_path) if filename.endswith(endswith_file)]

    def __get_image_shape(self, image_path):
        img = Image.open(image_path)
        w, h = img.size
        c = img.mode
        cc = ''
        if c == 'RGB':
            cc = 'RGB'
        elif c == 'RGBA':
            cc = 'RGBA'
        elif c == 'L':
            cc = 'Grayscale'
        elif c == '1':
            cc = 'Grayscale'
        else:
            cc = c
        return (w, h, c)

    def __select_data(self, file_path):
        processed_data = []
        list_class_name = self.json_file['datasets'][0]['names_class']

        if os.path.isfile(file_path):
            with open(file_path, 'r') as input_file:
                lines = input_file.readlines()

            for line in lines:
                parts = line.strip().split()
                label = parts[0] + ' ' + parts[1] + ' ' + parts[2]
                numbers = [round(float(num)) for num in parts[3:]]

                processed_line = {
                    "label": label,
                    "idlabel": int(list_class_name.index(label)),
                    "box_coordinates": {
                        'xmin': numbers[0],
                        'ymin': numbers[1],
                        'xmax': numbers[2],
                        'ymax': numbers[3]
                    }
                }
                processed_data.append(processed_line)
        else:
            print(f"{str(file_path)} not found!")

        return processed_data

    def process_files(self, file_paths, dataset_type):
        data = {}
        for path in tqdm(file_paths, desc=f"Processing {dataset_type} files"):
            img_dim = self.__get_image_shape(path)
            name_dir, name_input = os.path.split(path)
            name_dir = os.path.join(name_dir, 'Label' if dataset_type != 'test' else 'Annotations')
            id_name, type = name_input.split('.')
            name_label = id_name + '.txt'
            path_label = os.path.join(name_dir, name_label)
            sub_data = self.__select_data(path_label)

            if sub_data:  # Only add to JSON if sub_data is not empty
                data[str(id_name)] = {
                    "name": str(name_input),
                    "path": str(path),
                    "type": str(type),
                    "width": int(img_dim[0]),
                    "height": int(img_dim[1]),
                    "channel": str(img_dim[2]),
                    "boxes": sub_data
                }
                self.json_file['datasets'][0][dataset_type].append(data)

    def create_json_data(self):
        list_train_files = self.__select_files(self.path_train_images, '.jpg')
        list_valid_files = self.__select_files(self.path_valid_images, '.jpg')
        list_test_files = self.__select_files(self.path_test_images, '.jpg')

        # Process train files
        self.process_files(list_train_files, 'train')

        # Process valid files
        self.process_files(list_valid_files, 'valid')

        # Process test files
        self.process_files(list_test_files, 'test')

    def save_json_file(self):
        with open(os.path.join('/kaggle', 'working', self.name_json_file), 'wb') as f:
            f.write(orjson.dumps(self.json_file))

##  <font style="color:blue">1.2. Create JSON file of datasets</font>

In [7]:
%%time
transform_dataset = TransformDataset(name_json_file = 'datasets.json')
transform_dataset.create_json_data()
transform_dataset.save_json_file()


Processing valid files: 100%|██████████| 386/386 [00:07<00:00, 51.46it/s]
Processing test files: 0it [00:00, ?it/s]


CPU times: user 27.6 s, sys: 23.8 s, total: 51.5 s
Wall time: 2min 48s


#  <font style="color:green">2. Training [25 Points]</font> 



- **Write your training code in this section.**



- **You also have to share ([shared logs example](https://tensorboard.dev/experiment/JRtnsKbwTaq1ow6nPLPGeg)) the loss plot of your training using tensorboard.dev.** 



How to share TensorBoard logs using tensorboard.dev find [here](https://courses.opencv.org/courses/course-v1:OpenCV+OpenCV-106+2019_T1/courseware/b1c43ffe765246658e537109e188addb/d62572ec8bd344db9aeae81235ede618/4?activate_block_id=block-v1%3AOpenCV%2BOpenCV-106%2B2019_T1%2Btype%40vertical%2Bblock%40398b46ddcd5c465fa52cb4d572ba3229).

#  <font style="color:green">3. Inference [15 Points]</font> 



**You have to make predictions from your trained model on three images from the validation dataset.**



The plotted images should be similar to the following:



<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w10-p1.png'>







<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w10-p2.png'>







<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w10-p3.png'>






#  <font style="color:green">4. COCO Detection Evaluation [25 Points]</font> 



**You have to evaluate your detection model on COCO detection evaluation metric.**



For your reference here is the coco evaluation metric chart:





---



<img src="https://www.learnopencv.com/wp-content/uploads/2020/03/c3-w9-coco_metric.png">



---



#### <font style="color:red">The expected `AP` (primary challenge metric) is more than `0.5`.</font>



**The expected output should look similar to the following:**



```

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.550

 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.886

 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.629

 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.256

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.653

 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.627

 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.504

 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.629

 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.633

 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.380

 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.722

 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.704

```




# <font style="color:green">5. Run Inference on a Video [15 Points]</font>



#### [Download the Input Video](https://www.dropbox.com/s/g88o4dx18zpgn8k/projet3-input-video.mp4?dl=1)



**You have to run inference on a video.** 



You can download the video from [here](https://www.dropbox.com/s/g88o4dx18zpgn8k/projet3-input-video.mp4?dl=1).



#### <font style="color:red">Upload the output video on youtube and share the link. Do not upload the video in the lab.</font>

In [ ]:
from IPython.display import YouTubeVideo, display

video = YouTubeVideo("18HWHCevFdU", width=640, height=360)

display(video)

**Your output video should have a bounding box around the vehicle registration plate.**

In [ ]:
video = YouTubeVideo("5SgCuee7AMs", width=640, height=360)

display(video)

**You can use the following sample code to read and write a video.**

In [ ]:
def video_read_write(video_path):

    """

    Read video frames one-by-one, flip it, and write in the other video.

    video_path (str): path/to/video

    """

    video = cv2.VideoCapture(video_path)

    

    # Check if camera opened successfully

    if not video.isOpened(): 

        print("Error opening video file")

        return

    

    # create video writer

    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))

    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frames_per_second = video.get(cv2.CAP_PROP_FPS)

    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    

    output_fname = '{}_out.mp4'.format(os.path.splitext(video_path)[0])

    

    output_file = cv2.VideoWriter(

        filename=output_fname,

        # some installation of opencv may not support x264 (due to its license),

        # you can try other format (e.g. MPEG)

        fourcc=cv2.VideoWriter_fourcc(*"x264"),

        fps=float(frames_per_second),

        frameSize=(width, height),

        isColor=True,

    )

    

        

    i = 0

    while video.isOpened():

        ret, frame = video.read()

        if ret:

            

            output_file.write(frame[:, ::-1, :])

#             cv2.imwrite('anpd_out/frame_{}.png'.format(str(i).zfill(3)), frame[:, ::-1, :])

            i += 1

        else:

            break

        

    video.release()

    output_file.release()

    

    return